In [1]:
# sample data
import pandas as pd
import numpy as np

# Create sample financial data (in millions)
years = [2024, 2025, 2026, 2027, 2028]
data = {
    'Year': years,
    'Revenue': [14.67, 14.93, 15.20, 15.46, 15.77],  # Linear interpolation, 1.8% growth
    'Expenditures': [14.57, 15.06, 15.57, 16.10, 16.86],  # Linear interpolation, 3.7% growth
    'Fund_Balance': [5.37, 4.94, 4.57, 3.93, 2.97]  # Balance = Prev Balance + Revenue - Expenditures
}
df = pd.DataFrame(data)

# Expenditure breakdown (2024, in millions, scaled to total expenditures)
exp_data = {
    'Category': ['Salaries', 'Benefits', 'Purchased_Services', 'Supplies', 'Capital_Outlay'],
    'Amount': [14.57 * 0.55, 14.57 * 0.20, 14.57 * 0.15, 14.57 * 0.07, 14.57 * 0.03],  # FY 2024
    'Growth_Rate': [0.03, 0.05, 0.03, 0.03, 0.0]  # Annual growth from forecast
}
exp_df = pd.DataFrame(exp_data)

# Save for reproducibility
df.to_csv('flsd_forecast.csv', index=False)
exp_df.to_csv('flsd_expenditures.csv', index=False)

In [2]:
import sys
print(sys.executable)

c:\Users\steve\AppData\Local\Programs\Python\Python311\python.exe


In [3]:
# Model 1: Time-Series Forecast (ARIMA)
# Forecast revenue, expenditures, and fund balance through FY 2030 to quantify the deficit trajectory.
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data
df = pd.read_csv('flsd_forecast.csv')

# ARIMA forecasting function
def forecast_series(series, steps=2, order=(1,1,0)):
    model = ARIMA(series, order=order)
    fit = model.fit()
    forecast = fit.forecast(steps=steps)
    conf_int = fit.get_forecast(steps=steps).conf_int()
    return forecast, conf_int

# Forecast revenue and expenditures
forecast_years = [2029, 2030]
n_steps = len(forecast_years)

rev_forecast, rev_conf = forecast_series(df['Revenue'], steps=n_steps)
exp_forecast, exp_conf = forecast_series(df['Expenditures'], steps=n_steps)

# Calculate fund balance
last_balance = df['Fund_Balance'].iloc[-1]
forecast_balance = [last_balance]
for i in range(n_steps):
    balance = forecast_balance[-1] + rev_forecast.iloc[i] - exp_forecast.iloc[i]
    forecast_balance.append(balance)
forecast_balance = forecast_balance[1:]  # Remove initial value

# Combine historical and forecast data
all_years = df['Year'].tolist() + forecast_years
all_revenue = df['Revenue'].tolist() + rev_forecast.tolist()
all_expenditures = df['Expenditures'].tolist() + exp_forecast.tolist()
all_balance = df['Fund_Balance'].tolist() + forecast_balance

# Create Plotly visualization
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Scatter(x=all_years, y=all_revenue, name='Revenue ($M)', line=dict(color='blue')), secondary_y=False)
fig.add_trace(go.Scatter(x=all_years, y=all_expenditures, name='Expenditures ($M)', line=dict(color='red')), secondary_y=False)
fig.add_trace(go.Scatter(x=all_years, y=all_balance, name='Fund Balance ($M)', line=dict(color='green')), secondary_y=True)

# Add confidence intervals
fig.add_trace(go.Scatter(
    x=forecast_years + forecast_years[::-1],
    y=rev_conf['lower Revenue'].tolist() + rev_conf['upper Revenue'].tolist()[::-1],
    fill='toself',
    fillcolor='rgba(0,0,255,0.1)',
    line=dict(color='rgba(255,255,255,0)'),
    name='Revenue CI',
    showlegend=False
), secondary_y=False)
fig.add_trace(go.Scatter(
    x=forecast_years + forecast_years[::-1],
    y=exp_conf['lower Expenditures'].tolist() + exp_conf['upper Expenditures'].tolist()[::-1],
    fill='toself',
    fillcolor='rgba(255,0,0,0.1)',
    line=dict(color='rgba(255,255,255,0)'),
    name='Expenditures CI',
    showlegend=False
), secondary_y=False)

# Layout
fig.update_layout(
    title='FLSD Financial Forecast (FY 2024-2030)',
    xaxis_title='Year',
    yaxis_title='Revenue/Expenditures ($M)',
    yaxis2_title='Fund Balance ($M)',
    hovermode='x unified'
)
fig.show()

# Save as HTML for sharing
fig.write_html('flsd_forecast.html')

c:\Users\steve\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\steve\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


In [4]:
# Model 2: Expenditure Clustering Analysis
# Cluster expenditure categories to identify optimization targets (e.g., purchased services, supplies).

import pandas as pd
from sklearn.cluster import KMeans
import plotly.express as px

# Load expenditure data
exp_df = pd.read_csv('flsd_expenditures.csv')

# Prepare data for clustering
X = exp_df[['Amount', 'Growth_Rate']].values
kmeans = KMeans(n_clusters=3, random_state=42)
exp_df['Cluster'] = kmeans.fit_predict(X)

# Map clusters to labels
cluster_labels = {0: 'High-cost/Fixed', 1: 'Moderate-cost/Discretionary', 2: 'Low-cost/Stable'}
exp_df['Cluster_Label'] = exp_df['Cluster'].map(cluster_labels)

# Calculate percentage of total expenditures
exp_df['Percentage'] = (exp_df['Amount'] / exp_df['Amount'].sum()) * 100

# Create Plotly bar chart
fig = px.bar(
    exp_df,
    x='Category',
    y='Percentage',
    color='Cluster_Label',
    text='Percentage',
    title='FLSD Expenditure Breakdown (FY 2024)',
    labels={'Percentage': '% of Total Expenditures'},
    hover_data={'Amount': ':.2f', 'Growth_Rate': ':.2%'}
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='auto')
fig.add_annotation(
    x='Purchased_Services',
    y=exp_df[exp_df['Category'] == 'Purchased_Services']['Percentage'].iloc[0],
    text='Optimize: $100K-$200K savings',
    showarrow=True,
    arrowhead=2,
    ax=20,
    ay=-30
)
fig.update_layout(
    xaxis_title='Expenditure Category',
    yaxis_title='% of Total Expenditures',
    showlegend=True
)
fig.show()

# Save as HTML
fig.write_html('flsd_expenditures.html')

c:\Users\steve\AppData\Local\Programs\Python\Python311\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [5]:
# Plot 1: Fund Balance Trend with Zero Line
# Purpose: Highlight the declining fund balance and project when it might go negative
# Objective: Create urgency and demonstrate the need for immediate action

import plotly.graph_objects as go

fig = go.Figure()

# Add fund balance line
fig.add_trace(go.Scatter(
    x=all_years,
    y=all_balance,
    name='Fund Balance',
    line=dict(color='green', width=3)
))

# Add zero line
fig.add_hline(y=0, line_dash="dash", line_color="red",
              annotation_text="Zero Balance",
              annotation_position="bottom right")

# Add confidence interval for forecast
fig.add_trace(go.Scatter(
    x=forecast_years + forecast_years[::-1],
    y=[b + (r-e) for b,r,e in zip(forecast_balance, rev_forecast, exp_forecast)] + \
       [b - (r-e) for b,r,e in zip(forecast_balance[::-1], rev_forecast[::-1], exp_forecast[::-1])],
    fill='toself',
    fillcolor='rgba(0,255,0,0.1)',
    line=dict(color='rgba(255,255,255,0)'),
    name='Forecast CI',
    showlegend=False
))

fig.update_layout(
    title='FLSD Fund Balance Projection (FY 2024-2030)',
    xaxis_title='Year',
    yaxis_title='Fund Balance ($M)',
    hovermode='x unified',
    annotations=[
        dict(
            x=2030,
            y=-0.5,
            text="Projected Negative Balance",
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-40
        )
    ]
)

fig.show()
fig.write_html('flsd_balance_trend.html')

In [6]:
# Plot 2: Revenue vs Expenditures with Deficit Highlight
# Purpose: Show the growing gap between revenue and expenditures
# Objective: Demonstrate the structural deficit and its acceleration

fig = go.Figure()

# Add revenue bars
fig.add_trace(go.Bar(
    x=all_years,
    y=all_revenue,
    name='Revenue',
    marker_color='blue'
))

# Add expenditure bars
fig.add_trace(go.Bar(
    x=all_years,
    y=all_expenditures,
    name='Expenditures',
    marker_color='red'
))

# Add deficit line
deficits = [e - r for e, r in zip(all_expenditures, all_revenue)]
fig.add_trace(go.Scatter(
    x=all_years,
    y=deficits,
    name='Annual Deficit',
    line=dict(color='purple', width=2, dash='dot'),
    yaxis='y2'
))

fig.update_layout(
    title='FLSD Revenue vs Expenditures with Deficit Trend',
    xaxis_title='Year',
    yaxis_title='Amount ($M)',
    yaxis2=dict(
        title='Annual Deficit ($M)',
        overlaying='y',
        side='right'
    ),
    barmode='group',
    hovermode='x unified'
)

fig.show()
fig.write_html('flsd_revenue_expenditure_gap.html')

In [7]:
# Plot 3: Expenditure Breakdown (Donut Chart)
# Purpose: Show the distribution of expenditures across categories
# Objective: Highlight major cost centers and potential optimization areas

# Calculate percentages and format labels
exp_df['Percentage'] = (exp_df['Amount'] / exp_df['Amount'].sum()) * 100
exp_df['Label'] = exp_df.apply(lambda x: f"{x['Category']}<br>{x['Percentage']:.1f}%<br>(${x['Amount']:.2f}M)", axis=1)

fig = go.Figure(data=[go.Pie(
    labels=exp_df['Label'],
    values=exp_df['Amount'],
    hole=.4,
    marker_colors=px.colors.qualitative.Set3,
    textinfo='label',
    hovertemplate="<b>%{label}</b><br>" +
                  "Amount: $%{value:.2f}M<br>" +
                  "Growth Rate: %{customdata:.1%}<extra></extra>",
    customdata=exp_df['Growth_Rate']
)])

fig.update_layout(
    title='FLSD Expenditure Distribution (FY 2024)',
    annotations=[dict(
        text='Total:<br>$14.57M',
        x=0.5, y=0.5,
        font_size=20,
        showarrow=False
    )],
    showlegend=False
)

fig.show()
fig.write_html('flsd_expenditure_breakdown.html')

In [8]:
# Plot 4: Growth Rate Analysis
# Purpose: Compare growth rates across expenditure categories
# Objective: Identify areas of highest cost growth and potential savings opportunities

# Calculate future values for each category
years = range(2024, 2029)
future_values = []
for year in years:
    year_data = []
    for _, row in exp_df.iterrows():
        growth = (1 + row['Growth_Rate']) ** (year - 2024)
        future_value = row['Amount'] * growth
        year_data.append({
            'Year': year,
            'Category': row['Category'],
            'Amount': future_value,
            'Growth_Rate': row['Growth_Rate']
        })
    future_values.extend(year_data)

future_df = pd.DataFrame(future_values)

# Create the visualization
fig = go.Figure()

# Add bars for each category
for category in exp_df['Category'].unique():
    category_data = future_df[future_df['Category'] == category]
    fig.add_trace(go.Bar(
        name=category,
        x=category_data['Year'],
        y=category_data['Amount'],
        hovertemplate="<b>%{x}</b><br>" +
                     "Amount: $%{y:.2f}M<br>" +
                     "Growth Rate: %{customdata:.1%}<extra></extra>",
        customdata=[exp_df[exp_df['Category'] == category]['Growth_Rate'].iloc[0]] * len(category_data)
    ))

# Add growth rate annotations
for category in exp_df['Category'].unique():
    growth_rate = exp_df[exp_df['Category'] == category]['Growth_Rate'].iloc[0]
    if growth_rate > 0:
        fig.add_annotation(
            x=2024,
            y=future_df[(future_df['Category'] == category) & (future_df['Year'] == 2024)]['Amount'].iloc[0],
            text=f"{growth_rate:.1%} growth",
            showarrow=False,
            yshift=10,
            font=dict(size=10, color='red' if growth_rate > 0.03 else 'black')
        )

fig.update_layout(
    title='FLSD Expenditure Growth Projection (2024-2028)',
    xaxis_title='Year',
    yaxis_title='Amount ($M)',
    barmode='group',
    hovermode='x unified',
    showlegend=True,
    legend_title='Expenditure Category'
)

fig.show()
fig.write_html('flsd_growth_analysis.html')

In [9]:
# Plot 5: Cumulative Impact Analysis
# Purpose: Show the cumulative effect of annual deficits
# Objective: Demonstrate the long-term financial impact of current trends

# Calculate cumulative deficit
cumulative_deficit = []
running_total = 0
for rev, exp in zip(all_revenue, all_expenditures):
    running_total += (exp - rev)
    cumulative_deficit.append(running_total)

# Create the visualization
fig = go.Figure()

# Add area plot for cumulative deficit
fig.add_trace(go.Scatter(
    x=all_years,
    y=cumulative_deficit,
    fill='tozeroy',
    name='Cumulative Deficit',
    line=dict(color='red'),
    fillcolor='rgba(255,0,0,0.2)',
    hovertemplate="<b>%{x}</b><br>" +
                  "Cumulative Deficit: $%{y:.2f}M<extra></extra>"
))

# Add line for fund balance
fig.add_trace(go.Scatter(
    x=all_years,
    y=all_balance,
    name='Fund Balance',
    line=dict(color='green', width=2),
    hovertemplate="<b>%{x}</b><br>" +
                  "Fund Balance: $%{y:.2f}M<extra></extra>"
))

# Add zero line
fig.add_shape(
    type="line",
    x0=all_years[0],
    y0=0,
    x1=all_years[-1],
    y1=0,
    line=dict(color="black", width=1, dash="dash")
)

# Add annotations for key points
fig.add_annotation(
    x=all_years[-1],
    y=cumulative_deficit[-1],
    text=f"Total Deficit: ${cumulative_deficit[-1]:.2f}M",
    showarrow=True,
    arrowhead=2,
    ax=40,
    ay=-40
)

fig.add_annotation(
    x=all_years[-1],
    y=all_balance[-1],
    text=f"Final Balance: ${all_balance[-1]:.2f}M",
    showarrow=True,
    arrowhead=2,
    ax=40,
    ay=40
)

fig.update_layout(
    title='FLSD Cumulative Deficit Impact (2024-2030)',
    xaxis_title='Year',
    yaxis_title='Amount ($M)',
    hovermode='x unified',
    showlegend=True,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()
fig.write_html('flsd_cumulative_impact.html')

In [11]:
# Model 3: Purchased Services Analysis and Scenario Planning
# Analyze purchased services in detail and model different reduction scenarios

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create detailed purchased services breakdown (hypothetical data based on typical school district patterns)
purchased_services = {
    'Category': [
        'Professional Development',
        'Technology Services',
        'Maintenance Contracts',
        'Transportation Services',
        'Food Services',
        'Security Services',
        'Consulting Services',
        'Other Services'
    ],
    'Amount': [
        0.5,  # $500K
        0.4,  # $400K
        0.3,  # $300K
        0.2,  # $200K
        0.15, # $150K
        0.1,  # $100K
        0.1,  # $100K
        0.05  # $50K
    ],
    'Flexibility': [
        'High',
        'Medium',
        'Low',
        'Low',
        'Medium',
        'Low',
        'High',
        'High'
    ]
}

ps_df = pd.DataFrame(purchased_services)
ps_df['Percentage'] = (ps_df['Amount'] / ps_df['Amount'].sum()) * 100

# Create purchased services breakdown visualization
fig1 = go.Figure()
fig1.add_trace(go.Bar(
    x=ps_df['Category'],
    y=ps_df['Amount'],
    text=ps_df['Amount'].round(2),
    textposition='auto',
    marker_color='lightblue',
    name='Amount ($M)'
))

fig1.update_layout(
    title='Purchased Services Breakdown (FY 2024)',
    xaxis_title='Service Category',
    yaxis_title='Amount ($M)',
    showlegend=False,
    height=500
)

# Scenario Analysis
def calculate_scenario_impact(base_expenditures, reduction_percentages):
    scenarios = []
    for reduction in reduction_percentages:
        new_expenditures = base_expenditures.copy()
        new_expenditures['Purchased_Services'] *= (1 - reduction)
        new_expenditures['Supplies'] *= (1 - reduction)
        scenarios.append(new_expenditures)
    return scenarios

# Get base expenditures
base_expenditures = exp_df.set_index('Category')['Amount']

# Define reduction scenarios
reduction_percentages = [0.05, 0.10, 0.15, 0.20, 0.25]
scenarios = calculate_scenario_impact(base_expenditures, reduction_percentages)

# Calculate impact on fund balance
def calculate_fund_balance_impact(scenarios, revenue, years=5):
    impacts = []
    for scenario in scenarios:
        balance = 5.37  # Starting balance
        yearly_impacts = []
        for year in range(years):
            revenue_year = revenue * (1.018 ** year)  # 1.8% growth
            expenditure_year = scenario.sum() * (1.037 ** year)  # 3.7% growth
            balance = balance + revenue_year - expenditure_year
            yearly_impacts.append(balance)
        impacts.append(yearly_impacts)
    return impacts

# Calculate impacts
revenue_2024 = 14.67  # Base revenue
impacts = calculate_fund_balance_impact(scenarios, revenue_2024)

# Create scenario analysis visualization
fig2 = go.Figure()
years = list(range(2024, 2029))

for i, reduction in enumerate(reduction_percentages):
    fig2.add_trace(go.Scatter(
        x=years,
        y=impacts[i],
        name=f'{reduction*100}% Reduction',
        mode='lines+markers'
    ))

fig2.update_layout(
    title='Fund Balance Impact of Service & Supply Reductions',
    xaxis_title='Year',
    yaxis_title='Projected Fund Balance ($M)',
    showlegend=True,
    height=500
)

# Create summary table
summary_data = []
for i, reduction in enumerate(reduction_percentages):
    summary_data.append({
        'Reduction': f'{reduction*100}%',
        '2024 Savings': f'${(base_expenditures["Purchased_Services"] + base_expenditures["Supplies"]) * reduction:.2f}M',
        '2028 Balance': f'${impacts[i][-1]:.2f}M',
        'Balance Improvement': f'${impacts[i][-1] - impacts[0][-1]:.2f}M'
    })

summary_df = pd.DataFrame(summary_data)

# Display visualizations
fig1.show()
fig2.show()

# Display summary table
print("\nScenario Analysis Summary:")
print(summary_df.to_string(index=False))

# Save visualizations
fig1.write_html('purchased_services_breakdown.html')
fig2.write_html('reduction_scenarios.html')


Scenario Analysis Summary:
Reduction 2024 Savings 2028 Balance Balance Improvement
     5.0%       $0.16M       $3.83M              $0.00M
    10.0%       $0.32M       $4.69M              $0.86M
    15.0%       $0.48M       $5.55M              $1.73M
    20.0%       $0.64M       $6.42M              $2.59M
    25.0%       $0.80M       $7.28M              $3.45M


In [13]:
import pandas as pd
import plotly.express as px

# Project Purchased Services subcategories (FY 2024-2028)
years = [2024, 2025, 2026, 2027, 2028]
ps_trends = {
    'Year': years,
    'Utilities': [0.7 * (1.04)**i for i in range(5)],  # 4% growth
    'Transportation': [0.6 * (1.03)**i for i in range(5)],  # 3% growth
    'Technology': [0.5 * (1.05)**i for i in range(5)],  # 5% growth
    'Professional_Services': [0.3 * (1.02)**i for i in range(5)],  # 2% growth
    'Maintenance': [0.09 * (1.01)**i for i in range(5)]  # 1% growth
}
trend_df = pd.DataFrame(ps_trends)
trend_df['Total'] = trend_df[['Utilities', 'Transportation', 'Technology', 'Professional_Services', 'Maintenance']].sum(axis=1)

# Plotly line chart
fig = px.line(
    trend_df,
    x='Year',
    y=['Utilities', 'Transportation', 'Technology', 'Professional_Services', 'Maintenance', 'Total'],
    title='FLSD Purchased Services Growth (FY 2024-2028)',
    labels={'value': 'Cost ($M)', 'variable': 'Category'}
)
fig.add_annotation(
    x=2028,
    y=trend_df['Technology'].iloc[-1],
    text='High growth: $0.64M',
    showarrow=True,
    arrowhead=2
)
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Cost ($M)',
    hovermode='x unified'
)
fig.show()
fig.write_html('flsd_ps_trends.html')
fig.write_image('flsd_ps_trends.png')

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido
